In [1]:
import xarray as xr

# select location of interest
lat = 61.8474
lon = 24.2948

ds_era5land = xr.open_mfdataset("C:/STEMMUS_SCOPE_data/global/era5-land/*.nc").sel(latitude=lat, longitude=lon, method='nearest').compute()
ds_era5 = xr.open_mfdataset("C:/STEMMUS_SCOPE_data/global/era5/*.nc").sel(latitude=lat, longitude=lon, method='nearest').compute()

ds_era5land = ds_era5land.resample(time="1800S").interpolate('linear')
ds_era5land = ds_era5land.drop(['latitude', 'longitude'])

ds_era5 = ds_era5.resample(time="1800S").interpolate('linear')
ds_era5 = ds_era5.drop(['latitude', 'longitude'])

ds = xr.merge([ds_era5land, ds_era5])


KeyError: "not all values found in index 'latitude'. Try setting the `method` keyword argument (example: method='nearest')."

In [16]:
ds

<xarray.Dataset>
Dimensions:  (time: 1487)
Coordinates:
  * time     (time) datetime64[ns] 2016-01-01 ... 2016-01-31T23:00:00
Data variables:
    d2m      (time) float64 276.3 276.8 277.3 277.5 ... 282.3 282.4 282.5 282.6
    t2m      (time) float64 279.8 280.1 280.4 280.4 ... 283.3 283.4 283.5 283.6
    u10      (time) float64 2.421 2.588 2.754 2.854 ... 5.661 5.692 5.785 5.879
    v10      (time) float64 4.711 4.454 4.197 3.851 ... 0.3722 0.6098 0.8473
    mtpr     (time) float64 2.705e-05 3.618e-05 4.53e-05 ... 0.0001986 0.0002346
    sp       (time) float64 1.021e+05 1.021e+05 ... 1.008e+05 1.008e+05
    ssrd     (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    strd     (time) float64 1.025e+06 1.104e+06 ... 1.295e+06 1.293e+06
Attributes:
    Conventions:  CF-1.6
    history:      2023-01-05 14:26:51 UTC by era5cli 1.3.1: reanalysis-era5-l...

Calculate required variables

In [11]:
from PyStemmusScope import variable_conversion as vc

ds_ss = xr.Dataset()
ds_ss["wind_speed"] = (ds["u10"] ** 2 + ds["v10"] ** 2) ** 0.5
ds_ss["t_air_celcius"] = ds["t2m"] - 273.15  # K -> degC
ds_ss["precip"] = ds["mtpr"] / 10  # mm/s -> cm/s
ds_ss["p_surf"] = ds["sp"] / 1000  # Pa -> kPa
ds_ss["sw_down"] = ds["ssrd"] / 3600  # J * hr / m2 ->  W / m2
ds_ss["lw_down"] = ds["strd"] / 3600  # J * hr / m2 ->  W / m2

ds_ss["ea"] = vc.calculate_es(ds["d2m"] - 273.15)
ds_ss["vpd"] = vc.calculate_es(ds_ss["t_air_celcius"]) - ds_ss["ea"]
ds_ss["rh"] = ds_ss["ea"] / vc.calculate_es(ds_ss["t_air_celcius"])  
ds_ss["qair"] = vc.specific_humidity(ds_ss["ea"], ds_ss["p_surf"])